# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

In [4]:
import ast
from collections import Counter  # noqa
from multiprocessing import Pool, Queue, cpu_count
from pathlib import Path
from pprint import pprint
from typing import Mapping

import pandas as pd
from tqdm.notebook import tqdm

In [5]:
DATA_DIR = Path('data/')
SRC_DIR = Path('src/')
OUTPUT_DIR = Path('output/')

In [6]:
def dict_preview(dct: Mapping, n: int = 5) -> dict:
    return dict(list(dct.items())[:n])

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

In [7]:
CAP_PATH = DATA_DIR.joinpath('Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt')
PLAYER_PATH = DATA_DIR.joinpath('Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt')

In [8]:
%%file 'src/char_counts.py'
from collections import Counter


def char_counts(path) -> Counter:
    with open(path, encoding='cp1251') as f:
        return Counter(f.read().casefold())

Overwriting src/char_counts.py


In [9]:
from src.char_counts import char_counts

In [ ]:
%%timeit

char_counts(CAP_PATH)
char_counts(PLAYER_PATH)

In [ ]:
cap_cnt = char_counts(CAP_PATH)
player_cnt = char_counts(PLAYER_PATH)

In [ ]:
print('Prestuplenie i nakazanie:'.upper())
pprint(cap_cnt.most_common(8))
print('\nIgrok'.upper())
pprint(player_cnt.most_common(8))

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

In [ ]:
from src.char_counts import char_counts

In [ ]:
%%timeit

with Pool(2) as pool:
    pool.map(char_counts, [CAP_PATH, PLAYER_PATH])

In [ ]:
with Pool(2) as pool:
    results = pool.map(char_counts, [CAP_PATH, PLAYER_PATH])

In [ ]:
print('Prestuplenie i nakazanie:'.upper())
pprint(results[0].most_common(8))
print('\nIgrok'.upper())
pprint(results[1].most_common(8))

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```

In [ ]:
%%time

with pd.read_csv(
        DATA_DIR.joinpath('recipes_full.csv'),
        sep=',',
        chunksize=300000,
        usecols=['id', 'tags', 'n_steps']
) as reader:
    for i, chunk in tqdm(enumerate(reader), total=8):
        chunk['tags'] = chunk['tags'].apply(ast.literal_eval)
        chunk = chunk.explode('tags').rename(columns={'tags': 'tag'})
        chunk.to_csv(OUTPUT_DIR.joinpath(f'id_tag_nsteps_{i:0>2}.csv'), sep=';', index=False)

2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [ ]:
file_paths = [OUTPUT_DIR.joinpath(f'id_tag_nsteps_{i:0>2}.csv') for i in range(8)]

In [ ]:
def calc_mean_nsteps_by_tags(path) -> Mapping[str, float]:
    return pd.read_csv(path, sep=';').groupby('tag')['n_steps'].mean().to_dict()

In [ ]:
dict_preview(calc_mean_nsteps_by_tags(file_paths[0]))

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.

In [ ]:
def calc_mean_nsteps_by_tags(paths_list: list) -> Mapping[str, float]:
    results = [_calc_count_sum_nsteps_by_tags(path) for path in paths_list]
    return _aggregate_results(results).to_dict()


def _calc_count_sum_nsteps_by_tags(path) -> pd.DataFrame:
    return pd.read_csv(path, sep=';').groupby('tag')['n_steps'].agg(['sum', 'count'])


def _aggregate_results(results: list[pd.DataFrame]) -> pd.Series:
    res = sum(results)
    return res['sum'] / res['count']

In [ ]:
dict_preview(calc_mean_nsteps_by_tags(file_paths))

In [ ]:
%timeit calc_mean_nsteps_by_tags(file_paths)

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [ ]:
%%file 'src/task_4.py'
import pandas as pd


def worker(path) -> pd.DataFrame:
    return pd.read_csv(path, sep=';').groupby('tag')['n_steps'].agg(['sum', 'count'])


def aggregator(results: list[pd.DataFrame]) -> pd.Series:
    res = sum(results)
    return res['sum'] / res['count']

In [ ]:
from src.task_4 import worker, aggregator

In [ ]:
def calc_mean_nsteps_by_tags_concurrent(paths_list: list) -> Mapping[str, float]:
    with Pool(len(paths_list)) as pool:
        results = pool.map(worker, paths_list)
    return aggregator(list(results)).to_dict()

In [ ]:
dict_preview(calc_mean_nsteps_by_tags_concurrent(file_paths))

In [ ]:
from multiprocessing import Pool

%timeit calc_mean_nsteps_by_tags_concurrent(file_paths)

In [ ]:
from multiprocessing.pool import ThreadPool as Pool

%timeit calc_mean_nsteps_by_tags_concurrent(file_paths)

In [ ]:
from concurrent.futures import ThreadPoolExecutor as Pool

%timeit calc_mean_nsteps_by_tags_concurrent(file_paths)

5. (*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы.

In [ ]:
%%file 'src/task_5.py'
from functools import reduce
from multiprocessing import Queue, Process

import pandas as pd


class Worker(Process):

    def __init__(self, tasks_queue: Queue, results_queue: Queue):
        self._tasks_queue = tasks_queue
        self._results_queue = results_queue
        super(Worker, self).__init__()

    def run(self) -> None:
        while not self._tasks_queue.empty():
            task = self._tasks_queue.get()
            self._results_queue.put(task())


class Task:

    def __init__(self, path):
        self._path = path

    def __call__(self) -> pd.DataFrame:
        return pd.read_csv(self._path, sep=';').groupby('tag')['n_steps'].agg(['sum', 'count'])


def aggregate(results_queue: Queue, qsize: int) -> dict[str, float]:
    initial = results_queue.get()
    res = reduce(lambda a, b: a + b, lazy_queue_get(results_queue, qsize - 1), initial)
    return (res['sum'] / res['count']).to_dict()


def lazy_queue_get(queue: Queue, qsize: int):
    for i in range(qsize):
        yield queue.get()

In [ ]:
from src.task_5 import Worker, Task, aggregate

In [ ]:
def calc_mean_nsteps_by_tags_process_sync(paths_list: list) -> dict[str, float]:
    tasks_queue = Queue()
    results_queue = Queue()

    for path in paths_list:
        tasks_queue.put(Task(path))

    for i in range(int(cpu_count() / 2 / 2)):
        Worker(tasks_queue, results_queue).start()

    return aggregate(results_queue, len(paths_list))

In [ ]:
dict_preview(calc_mean_nsteps_by_tags_process_sync(file_paths))

In [ ]:
%timeit dict_preview(calc_mean_nsteps_by_tags_process_sync(file_paths))